# Advent of Code 2021 - Day 1
> My solution and thoughts

- toc: false
- badges: false
- comments: false
- categories: [advent-of-code]

# Advent of Code
I have, on every occasion that I have tried, completely failed to follow through on the [Advent of Code](https://adventofcode.com/). This is the year that I change that!

I like Python, so I'm going to be presenting various solutions with Python as a way to play with it again. My goal is to explore the various approaches that one might take, and to think through things from a "first principles" kind of mindset.

So with that said, let's look at the first day! I'm not going to copy over the text from the problems, but I will state them in my own words.

## Part 1
For the first part, we're given a list of numbers, and we are looking to determine how many of the numbers are larger than the one before. There's some story around it, and I appreciate the stories, but this is the gist.

The example is, if we're given:
```
199
200
208
210
200
207
240
269
260
263
```
then we would note that:
```
199 (N/A - no previous measurement)
200 (increased)
208 (increased)
210 (increased)
200 (decreased)
207 (increased)
240 (increased)
269 (increased)
260 (decreased)
263 (increased)
```

Ok, so we need to get our input and preferably get it into numeric data types. I'll start out with a totally naive approach.

In [12]:
with open('../assets/inputs/aoc/2021/day_1.txt', 'r') as data_file:
    data = [int(point.strip()) for point in data_file.readlines()]

We strip the strings and then cast them to integer. This is a dandy way to get a list of values, but now we need to determine how they're bigger. We could do this in several ways, but the simplest is to iterate the list and make the comparison, storing a count of the values that are, in fact, larger than their predecessor.

In [15]:
count = 0
for i, n in enumerate(data):
    if n > data[i - 1]:
        count += 1
print(count)

1521


Success! Your data is probably different, but this is the correct answer. We pass the list once, so the complexity here is `O(n)` on the input. Naively, we would expect that to be the limit for how good this could be. But what about the code itself?

Personally, I don't like to have state (`count`) just chilling and getting mutated. That's a recipe for mixups.

So let's look at what we're doing: applying a rule across an iterable and accumulating a result. That sounds an awful lot like a [reduce](https://docs.python.org/3/library/functools.html#functools.reduce). Trouble is, we don't really care about the accumulated value in the rule. We only care about the pairwise elements...

If we don't care about running through the list twice, then we can use a map in the form of a list comprehension to do this pretty effectively:

In [28]:
count = sum([int(n > data[i - 1]) for i, n in enumerate(data)])

print(count)

1521


Personally, I like this better. Let's go ahead and make a function out of it for convenience though.

In [29]:
def count_increase_depths(data):
    return sum([int(n > data[i - 1]) for i, n in enumerate(data)])

## Part 2
So part two switches up the approach. We're going to use the same data, but instead of looking at a single value and it's predecessor, we want to create a sliding window of sums for sets of 3 values. This is pretty easy to achieve:

In [30]:
windows = [data[i:i + 3] for i in range(len(data))]

display(windows[:10])

[[173, 175, 171],
 [175, 171, 177],
 [171, 177, 179],
 [177, 179, 177],
 [179, 177, 174],
 [177, 174, 177],
 [174, 177, 178],
 [177, 178, 185],
 [178, 185, 189],
 [185, 189, 195]]

Looking at the first ten windows, it looks like we've built them correctly, but it always pays to look at the front _and_ the back of a data set.

In [27]:
display(windows[1990:])

[[7063, 7065, 7066],
 [7065, 7066, 7071],
 [7066, 7071, 7079],
 [7071, 7079, 7092],
 [7079, 7092, 7102],
 [7092, 7102, 7118],
 [7102, 7118, 7115],
 [7118, 7115, 7121],
 [7115, 7121],
 [7121]]

Ah, trailing data from the windows. We can address that by altering the way we build them:

In [26]:
better_windows = [data[i:i + 3] for i in range(len(data) - 2)]

display(better_windows[1990:])

[[7063, 7065, 7066],
 [7065, 7066, 7071],
 [7066, 7071, 7079],
 [7071, 7079, 7092],
 [7079, 7092, 7102],
 [7092, 7102, 7118],
 [7102, 7118, 7115],
 [7118, 7115, 7121]]

Now, we could generalize this. The window distance itself is related to how many trailing values we end up with. If we defined the window as 4 instead of 3 we'd have gotten 3 trailing values. This could be a helpful function in the future, so let's go ahead and write it:

In [25]:
def windowed(iterable, window_size):
    adjust = window_size - 1
    windowing_range = range(len(iterable) - adjust)
    
    return [iterable[i:i + window_size] for i in windowing_range]

display(windowed(data, 3)[1990:])

[[7063, 7065, 7066],
 [7065, 7066, 7071],
 [7066, 7071, 7079],
 [7071, 7079, 7092],
 [7079, 7092, 7102],
 [7092, 7102, 7118],
 [7102, 7118, 7115],
 [7118, 7115, 7121]]

I like it! Now we basically need to run the same count algorithm on this, but after summing them.

In [31]:
windowed_data = windowed(data, 3)
summed = [sum(window) for window in windowed_data]

display(count_increase_depths(summed))

1543

## Conclusion
The first day wasn't a terribly difficult challenge, but it did get me thinking in Python again! I'm really looking forward to doing the rest of the days this year. This should be a lot of fun.